In [5]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *
from src.validation import validate_jumpstart_cube, display_validate_results
from src.coherence import analyze_deck_theme_coherence_enhanced
from src.improve import apply_swap

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
cube_df = pd.read_csv('JumpstartCube_ThePauperCube_ULTIMATE_Final.csv')

# Optimise

In [2]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck

# import importlib
# import src.process
# importlib.reload(src.process)

# from src.process import optimize_deck_coherence, clear_swap_history

# # Clear any previous swap history to prevent issues with oscillation
# clear_swap_history()

# coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
# total_coherence = sum(result['overall_coherence'] for result in coherence.values())

# improvement = True
# iteration = 0
# max_iterations = 20  # Prevent infinite loops

# while improvement and iteration < max_iterations:
#     prev_total_coherence = total_coherence
#     cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df)
#     coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
#     total_coherence = sum(result['overall_coherence'] for result in coherence.values())
#     improvement = total_coherence > prev_total_coherence
#     iteration += 1
#     print(f"Iteration {iteration}: Total coherence: {total_coherence:.2f} (improvement: {total_coherence - prev_total_coherence:.2f})")
    
#     if not improvement:
#         print("No further improvements found - optimization complete!")
#     elif iteration >= max_iterations:
#         print("Maximum iterations reached - stopping optimization")

# Save to file

In [ ]:
# from src.export import export_cube_to_csv

# export_cube_to_csv(cube_df, oracle_df, 'JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv')

Exporting cube to JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv...
✅ Successfully exported 390 cards to JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv

📊 Export Summary:
Total cards: 390
Number of decks: 30

Deck breakdown:
  Azorius Evasion/Flying: 13 cards
  Black Aggro: 13 cards
  White Equipment: 13 cards
  White Control: 13 cards
  White Aggro: 13 cards
  Simic Control: 13 cards
  Selesnya Control: 13 cards
  Red Artifacts: 13 cards
  Red Burn: 13 cards
  Rakdos Burn/Damage: 13 cards
  ... and 20 more decks


'JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv'

# Analysis

In [4]:
# from src.coherence import display_coherence_analysis_enhanced


# display_coherence_analysis_enhanced(analyze_deck_theme_coherence_enhanced(cube_df, oracle_df))

In [11]:
# Let's examine the Black Graveyard deck
black_graveyard = cube_df[cube_df['Tags'].str.contains('Black Graveyard', na=False)]
print("Current Black Graveyard deck:")
for idx, row in black_graveyard.iterrows():
    print(f"- {row['Name']} ({row['Type']}) - CMC {row['CMC']} - {row['Color Identity']}")
print(f"\nTotal cards in Black Graveyard: {len(black_graveyard)}")

# Find unassigned cards
unassigned = cube_df[cube_df['Tags'].str.contains('Unassigned', na=False)]
print(f"\nUnassigned cards ({len(unassigned)} total):")
for idx, row in unassigned.iterrows():
    print(f"- {row['Name']} ({row['Type']}) - CMC {row['CMC']} - {row['Color Identity']}")

# Let's also see if Scrapwork Mutt is indeed in the Black Graveyard deck
scrapwork_mutt = cube_df[cube_df['Name'].str.contains('Scrapwork Mutt', na=False)]
if len(scrapwork_mutt) > 0:
    print(f"\nScrapwork Mutt info:")
    for idx, row in scrapwork_mutt.iterrows():
        print(f"- {row['Name']} - Tag: {row['Tags']} - Type: {row['Type']} - CMC: {row['CMC']} - Color: {row['Color Identity']}")
else:
    print("\nScrapwork Mutt not found in the cube")

Current Black Graveyard deck:
- Blood Fountain (Artifact) - CMC 1 - B
- Crypt Rats (Creature - Rat) - CMC 3 - B
- Faceless Butcher (Creature - Nightmare Horror) - CMC 4 - B
- First-Sphere Gargantua (Creature - Horror) - CMC 6 - B
- Grim Bauble (Artifact) - CMC 1 - B
- Gurmag Angler (Creature - Zombie Fish) - CMC 7 - B
- Nested Shambler (Creature - Zombie) - CMC 1 - B
- Nezumi Linkbreaker (Creature - Rat Warlock) - CMC 1 - B
- Night's Whisper (Sorcery) - CMC 2 - B
- Okiba-Gang Shinobi (Creature - Rat Ninja) - CMC 5 - B
- Putrid Goblin (Creature - Zombie Goblin) - CMC 2 - B
- Tragic Slip (Instant) - CMC 1 - B
- Scrapwork Mutt (Artifact Creature - Dog) - CMC 2 - nan

Total cards in Black Graveyard: 13

Unassigned cards (0 total):

Scrapwork Mutt info:
- Scrapwork Mutt - Tag: Black Graveyard - Type: Artifact Creature - Dog - CMC: 2 - Color: nan


In [13]:
# First, let's check the column names in both dataframes
print("Cube_df columns:", cube_df.columns.tolist())
print("Oracle_df columns:", oracle_df.columns.tolist())
print("\nCube_df shape:", cube_df.shape)
print("Oracle_df shape:", oracle_df.shape)

Cube_df columns: ['Name', 'Set', 'Collector Number', 'Rarity', 'Color Identity', 'Type', 'Mana Cost', 'CMC', 'Power', 'Toughness', 'Tags']
Oracle_df columns: ['name', 'CMC', 'Type', 'Color', 'Color Category', 'Oracle Text', 'tags', 'MTGO ID', 'Power', 'Toughness']

Cube_df shape: (390, 11)
Oracle_df shape: (450, 10)


In [14]:
# Find cards that are in oracle_df but not in cube_df (truly unassigned)
cube_card_names = set(cube_df['Name'].str.strip().str.lower())
oracle_card_names = set(oracle_df['name'].str.strip().str.lower())

truly_unassigned_names = oracle_card_names - cube_card_names
truly_unassigned = oracle_df[oracle_df['name'].str.strip().str.lower().isin(truly_unassigned_names)]

print(f"\nCards in oracle but not in cube ({len(truly_unassigned)} total):")

# Filter for cards that would fit Black Graveyard theme (black cards, artifacts, cards with graveyard synergies)
black_graveyard_candidates = truly_unassigned[
    (truly_unassigned['Color Category'].str.contains('Black', na=False)) |
    (truly_unassigned['Type'].str.contains('Artifact', na=False)) |
    (truly_unassigned['Oracle Text'].str.contains('graveyard|grave|return.*from.*graveyard|sacrifice|dies|death|zombie|skeleton', case=False, na=False))
]

print(f"\nPotential Black Graveyard candidates ({len(black_graveyard_candidates)} total):")
for idx, row in black_graveyard_candidates.iterrows():
    print(f"- {row['name']} ({row['Type']}) - CMC {row['CMC']} - {row['Color Category']}")
    if pd.notna(row['Oracle Text']) and len(row['Oracle Text']) > 0:
        # Show a snippet of oracle text for graveyard-related abilities
        oracle_snippet = row['Oracle Text'][:100] + "..." if len(row['Oracle Text']) > 100 else row['Oracle Text']
        print(f"  Oracle: {oracle_snippet}")
    print()


Cards in oracle but not in cube (60 total):

Potential Black Graveyard candidates (41 total):
- Prismatic Strands (Instant) - CMC 3 - White
  Oracle: Prevent all damage that sources of the color of your choice would deal this turn. | Flashback—Tap an...

- Mystic Sanctuary (Land - Island) - CMC 0 - Blue
  Oracle: ({T}: Add {U}.) | This land enters tapped unless you control three or more other Islands. | When thi...

- Basilica Screecher (Creature - Bat) - CMC 2 - Black
  Oracle: Flying | Extort (Whenever you cast a spell, you may pay {W/B}. If you do, each opponent loses 1 life...

- Feed the Swarm (Sorcery) - CMC 2 - Black
  Oracle: Destroy target creature or enchantment an opponent controls. You lose life equal to that permanent's...

- Arms of Hadar (Sorcery) - CMC 4 - Black
  Oracle: Creatures target player controls get -2/-2 until end of turn.

- Clawing Torment (Enchantment - Aura) - CMC 1 - Black
  Oracle: Enchant artifact or creature | As long as enchanted permanent is a creat

In [15]:
# Let's focus on the most relevant candidates with specific graveyard themes
print("=== TOP BLACK GRAVEYARD CANDIDATES ===\n")

# Score candidates based on graveyard synergy
graveyard_keywords = ['graveyard', 'grave', 'return.*from.*graveyard', 'sacrifice', 'dies', 'when.*dies', 'death', 'zombie', 'skeleton', 'reanimate', 'unearth', 'dredge', 'flashback']

def score_graveyard_synergy(oracle_text):
    if pd.isna(oracle_text):
        return 0
    score = 0
    oracle_lower = oracle_text.lower()
    for keyword in graveyard_keywords:
        if keyword in oracle_lower:
            score += 1
    return score

# Add graveyard synergy scores
black_graveyard_candidates['graveyard_score'] = black_graveyard_candidates['Oracle Text'].apply(score_graveyard_synergy)

# Filter for cards with CMC 1-6 (similar to current deck range) and good graveyard synergy
top_candidates = black_graveyard_candidates[
    (black_graveyard_candidates['CMC'] <= 6) & 
    (black_graveyard_candidates['graveyard_score'] > 0)
].sort_values(['graveyard_score', 'CMC'], ascending=[False, True])

print(f"Top {min(10, len(top_candidates))} candidates (sorted by graveyard synergy):")
for idx, row in top_candidates.head(10).iterrows():
    print(f"\n{row['name']} - CMC {row['CMC']} - {row['Color Category']} - Score: {row['graveyard_score']}")
    print(f"Type: {row['Type']}")
    if pd.notna(row['Oracle Text']):
        print(f"Oracle: {row['Oracle Text']}")

# Also show some low-CMC options that might fit the curve
print("\n=== LOW CMC OPTIONS (1-2 mana) ===")
low_cmc_candidates = black_graveyard_candidates[
    (black_graveyard_candidates['CMC'] <= 2) & 
    (black_graveyard_candidates['Color Category'].str.contains('Black', na=False))
].sort_values('CMC')

for idx, row in low_cmc_candidates.head(5).iterrows():
    print(f"\n{row['name']} - CMC {row['CMC']} - {row['Color Category']}")
    print(f"Type: {row['Type']}")
    if pd.notna(row['Oracle Text']):
        oracle_snippet = row['Oracle Text'][:150] + "..." if len(row['Oracle Text']) > 150 else row['Oracle Text']
        print(f"Oracle: {oracle_snippet}")

=== TOP BLACK GRAVEYARD CANDIDATES ===

Top 10 candidates (sorted by graveyard synergy):

Haunted Fengraf - CMC 0 - Lands - Score: 3
Type: Land
Oracle: {T}: Add {C}. | {3}, {T}, Sacrifice this land: Return a creature card at random from your graveyard to your hand.

Prismatic Strands - CMC 3 - White - Score: 3
Type: Instant
Oracle: Prevent all damage that sources of the color of your choice would deal this turn. | Flashback—Tap an untapped white creature you control. (You may cast this card from your graveyard for its flashback cost. Then exile it.)

Mystic Sanctuary - CMC 0 - Blue - Score: 2
Type: Land - Island
Oracle: ({T}: Add {U}.) | This land enters tapped unless you control three or more other Islands. | When this land enters untapped, you may put target instant or sorcery card from your graveyard on top of your library.

Witch's Cottage - CMC 0 - Black - Score: 2
Type: Land - Swamp
Oracle: ({T}: Add {B}.) | This land enters tapped unless you control three or more other Swamps. |

/var/folders/lf/f83j_kbn0qdbdd7j8rwyj0m80000gn/T/ipykernel_41511/273263262.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  black_graveyard_candidates['graveyard_score'] = black_graveyard_candidates['Oracle Text'].apply(score_graveyard_synergy)


In [16]:
# Create a concise summary of recommendations
print("=== RECOMMENDATION SUMMARY ===")
print("Current Black Graveyard deck has 13 cards with CMC range 1-7")
print("Looking for cards that synergize with graveyard themes...\n")

# Get the best candidates
top_5 = top_candidates.head(5)
print("TOP 5 RECOMMENDATIONS:")
for i, (idx, row) in enumerate(top_5.iterrows(), 1):
    print(f"{i}. {row['name']} (CMC {row['CMC']}) - Graveyard Synergy Score: {row['graveyard_score']}")
    
print("\nDetailed analysis available in the output above. Based on the Black Graveyard theme,")
print("I recommend adding a card that:")
print("- Has strong graveyard synergies (death triggers, graveyard interaction)")
print("- Fits the mana curve (preferably CMC 1-4 for balance)")
print("- Is either Black or colorless/artifact")
print("- Synergizes with existing cards like Gurmag Angler, First-Sphere Gargantua, etc.")

=== RECOMMENDATION SUMMARY ===
Current Black Graveyard deck has 13 cards with CMC range 1-7
Looking for cards that synergize with graveyard themes...

TOP 5 RECOMMENDATIONS:
1. Haunted Fengraf (CMC 0) - Graveyard Synergy Score: 3
2. Prismatic Strands (CMC 3) - Graveyard Synergy Score: 3
3. Mystic Sanctuary (CMC 0) - Graveyard Synergy Score: 2
4. Witch's Cottage (CMC 0) - Graveyard Synergy Score: 2
5. Conduit Pylons (CMC 0) - Graveyard Synergy Score: 2

Detailed analysis available in the output above. Based on the Black Graveyard theme,
I recommend adding a card that:
- Has strong graveyard synergies (death triggers, graveyard interaction)
- Fits the mana curve (preferably CMC 1-4 for balance)
- Is either Black or colorless/artifact
- Synergizes with existing cards like Gurmag Angler, First-Sphere Gargantua, etc.


In [17]:
# Implement the addition of Haunted Fengraf to Black Graveyard deck
print("=== IMPLEMENTING RECOMMENDATION ===")
print("Adding Haunted Fengraf to the Black Graveyard deck...\n")

# Find Haunted Fengraf in the oracle_df
haunted_fengraf = oracle_df[oracle_df['name'].str.contains('Haunted Fengraf', case=False, na=False)]

if len(haunted_fengraf) > 0:
    print("Found Haunted Fengraf in oracle:")
    for idx, row in haunted_fengraf.iterrows():
        print(f"- {row['name']} ({row['Type']}) - CMC {row['CMC']}")
        print(f"  Oracle Text: {row['Oracle Text']}")
    
    # Get the card details
    card_row = haunted_fengraf.iloc[0]
    
    # Create new row for cube_df (matching the cube_df column structure)
    new_row = {
        'Name': card_row['name'],
        'Set': 'Unknown',  # We don't have this info in oracle_df
        'Collector Number': 'Unknown',  # We don't have this info in oracle_df
        'Rarity': 'Unknown',  # We don't have this info in oracle_df
        'Color Identity': card_row['Color Category'] if pd.notna(card_row['Color Category']) else 'Colorless',
        'Type': card_row['Type'],
        'Mana Cost': '',  # Land typically has no mana cost
        'CMC': card_row['CMC'],
        'Power': card_row['Power'] if pd.notna(card_row['Power']) else '',
        'Toughness': card_row['Toughness'] if pd.notna(card_row['Toughness']) else '',
        'Tags': 'Black Graveyard'
    }
    
    # Add the new row to cube_df
    cube_df = pd.concat([cube_df, pd.DataFrame([new_row])], ignore_index=True)
    
    print(f"\n✅ Successfully added {card_row['name']} to the Black Graveyard deck!")
    print(f"New cube size: {len(cube_df)} cards")
    
    # Verify the addition
    updated_black_graveyard = cube_df[cube_df['Tags'].str.contains('Black Graveyard', na=False)]
    print(f"Updated Black Graveyard deck now has {len(updated_black_graveyard)} cards")
    
else:
    print("❌ Haunted Fengraf not found in oracle_df")
    print("Available land candidates with graveyard synergy:")
    land_candidates = black_graveyard_candidates[black_graveyard_candidates['Type'].str.contains('Land', na=False)]
    for idx, row in land_candidates.head(3).iterrows():
        print(f"- {row['name']} (Score: {row['graveyard_score']})")

=== IMPLEMENTING RECOMMENDATION ===
Adding Haunted Fengraf to the Black Graveyard deck...

Found Haunted Fengraf in oracle:
- Haunted Fengraf (Land) - CMC 0
  Oracle Text: {T}: Add {C}. | {3}, {T}, Sacrifice this land: Return a creature card at random from your graveyard to your hand.

✅ Successfully added Haunted Fengraf to the Black Graveyard deck!
New cube size: 391 cards
Updated Black Graveyard deck now has 14 cards


In [18]:
# Verify the updated Black Graveyard deck
print("=== UPDATED BLACK GRAVEYARD DECK ===")
updated_black_graveyard = cube_df[cube_df['Tags'].str.contains('Black Graveyard', na=False)]

print(f"Black Graveyard deck now contains {len(updated_black_graveyard)} cards:")
for idx, row in updated_black_graveyard.iterrows():
    print(f"- {row['Name']} ({row['Type']}) - CMC {row['CMC']} - {row['Color Identity']}")

print(f"\n📊 Mana Curve Analysis:")
cmc_counts = updated_black_graveyard['CMC'].value_counts().sort_index()
for cmc, count in cmc_counts.items():
    print(f"CMC {cmc}: {count} cards")

print(f"\n🎯 The addition of Haunted Fengraf provides:")
print("- Late-game graveyard value without affecting the mana curve")
print("- Synergy with creature-heavy strategy (works with Gurmag Angler, First-Sphere Gargantua, etc.)")
print("- Additional utility land that doesn't compete for spell slots")
print("- Perfect thematic fit for the graveyard-focused deck")

=== UPDATED BLACK GRAVEYARD DECK ===
Black Graveyard deck now contains 14 cards:
- Blood Fountain (Artifact) - CMC 1 - B
- Crypt Rats (Creature - Rat) - CMC 3 - B
- Faceless Butcher (Creature - Nightmare Horror) - CMC 4 - B
- First-Sphere Gargantua (Creature - Horror) - CMC 6 - B
- Grim Bauble (Artifact) - CMC 1 - B
- Gurmag Angler (Creature - Zombie Fish) - CMC 7 - B
- Nested Shambler (Creature - Zombie) - CMC 1 - B
- Nezumi Linkbreaker (Creature - Rat Warlock) - CMC 1 - B
- Night's Whisper (Sorcery) - CMC 2 - B
- Okiba-Gang Shinobi (Creature - Rat Ninja) - CMC 5 - B
- Putrid Goblin (Creature - Zombie Goblin) - CMC 2 - B
- Tragic Slip (Instant) - CMC 1 - B
- Scrapwork Mutt (Artifact Creature - Dog) - CMC 2 - nan
- Haunted Fengraf (Land) - CMC 0 - Lands

📊 Mana Curve Analysis:
CMC 0: 1 cards
CMC 1: 5 cards
CMC 2: 3 cards
CMC 3: 1 cards
CMC 4: 1 cards
CMC 5: 1 cards
CMC 6: 1 cards
CMC 7: 1 cards

🎯 The addition of Haunted Fengraf provides:
- Late-game graveyard value without affecting t

In [19]:
# Remove Scrapwork Mutt from Black Graveyard deck
print("=== REMOVING SCRAPWORK MUTT ===")
print("Removing Scrapwork Mutt from the Black Graveyard deck...\n")

# Find Scrapwork Mutt in the current cube
scrapwork_mutt_rows = cube_df[cube_df['Name'].str.contains('Scrapwork Mutt', case=False, na=False)]

if len(scrapwork_mutt_rows) > 0:
    print("Found Scrapwork Mutt in cube:")
    for idx, row in scrapwork_mutt_rows.iterrows():
        print(f"- {row['Name']} - Tag: {row['Tags']} - Type: {row['Type']} - CMC: {row['CMC']}")
    
    # Get the original cube size
    original_size = len(cube_df)
    
    # Remove Scrapwork Mutt from the cube
    cube_df = cube_df[~cube_df['Name'].str.contains('Scrapwork Mutt', case=False, na=False)]
    
    new_size = len(cube_df)
    removed_count = original_size - new_size
    
    print(f"\n✅ Successfully removed {removed_count} instance(s) of Scrapwork Mutt!")
    print(f"Cube size reduced from {original_size} to {new_size} cards")
    
    # Verify the removal from Black Graveyard deck
    updated_black_graveyard = cube_df[cube_df['Tags'].str.contains('Black Graveyard', na=False)]
    print(f"Black Graveyard deck now has {len(updated_black_graveyard)} cards")
    
else:
    print("❌ Scrapwork Mutt not found in the cube")

=== REMOVING SCRAPWORK MUTT ===
Removing Scrapwork Mutt from the Black Graveyard deck...

Found Scrapwork Mutt in cube:
- Scrapwork Mutt - Tag: Black Graveyard - Type: Artifact Creature - Dog - CMC: 2

✅ Successfully removed 1 instance(s) of Scrapwork Mutt!
Cube size reduced from 391 to 390 cards
Black Graveyard deck now has 13 cards


In [20]:
# Verify the updated Black Graveyard deck after removal
print("=== UPDATED BLACK GRAVEYARD DECK (After Removal) ===")
final_black_graveyard = cube_df[cube_df['Tags'].str.contains('Black Graveyard', na=False)]

print(f"Black Graveyard deck now contains {len(final_black_graveyard)} cards:")
for idx, row in final_black_graveyard.iterrows():
    print(f"- {row['Name']} ({row['Type']}) - CMC {row['CMC']} - {row['Color Identity']}")

print(f"\n📊 Updated Mana Curve Analysis:")
cmc_counts_final = final_black_graveyard['CMC'].value_counts().sort_index()
for cmc, count in cmc_counts_final.items():
    print(f"CMC {cmc}: {count} cards")

print(f"\n🔄 Changes Summary:")
print("- ❌ Removed: Scrapwork Mutt (CMC 2 Artifact Creature)")
print("- ✅ Added: Haunted Fengraf (CMC 0 Land with graveyard synergy)")
print("- 📈 Result: Better thematic coherence with graveyard focus")
print("- 🎯 The deck now has stronger graveyard synergies and utility land support")

print(f"\n📊 Final Stats:")
print(f"- Total cube size: {len(cube_df)} cards") 
print(f"- Black Graveyard deck size: {len(final_black_graveyard)} cards")
print(f"- Mana curve: {cmc_counts_final.index.min()}-{cmc_counts_final.index.max()} CMC")

=== UPDATED BLACK GRAVEYARD DECK (After Removal) ===
Black Graveyard deck now contains 13 cards:
- Blood Fountain (Artifact) - CMC 1 - B
- Crypt Rats (Creature - Rat) - CMC 3 - B
- Faceless Butcher (Creature - Nightmare Horror) - CMC 4 - B
- First-Sphere Gargantua (Creature - Horror) - CMC 6 - B
- Grim Bauble (Artifact) - CMC 1 - B
- Gurmag Angler (Creature - Zombie Fish) - CMC 7 - B
- Nested Shambler (Creature - Zombie) - CMC 1 - B
- Nezumi Linkbreaker (Creature - Rat Warlock) - CMC 1 - B
- Night's Whisper (Sorcery) - CMC 2 - B
- Okiba-Gang Shinobi (Creature - Rat Ninja) - CMC 5 - B
- Putrid Goblin (Creature - Zombie Goblin) - CMC 2 - B
- Tragic Slip (Instant) - CMC 1 - B
- Haunted Fengraf (Land) - CMC 0 - Lands

📊 Updated Mana Curve Analysis:
CMC 0: 1 cards
CMC 1: 5 cards
CMC 2: 2 cards
CMC 3: 1 cards
CMC 4: 1 cards
CMC 5: 1 cards
CMC 6: 1 cards
CMC 7: 1 cards

🔄 Changes Summary:
- ❌ Removed: Scrapwork Mutt (CMC 2 Artifact Creature)
- ✅ Added: Haunted Fengraf (CMC 0 Land with graveya